In [59]:
google_document_xlsx = "https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx"

In [ ]:
!wget {google_document_xlsx} -O /tmp/prices_scans.xlsx

--2025-04-17 13:04:35--  https://docs.google.com/spreadsheets/d/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD/export?format=xlsx
Resolving docs.google.com (docs.google.com)... 216.58.211.238, 2a00:1450:4026:808::200e
Connecting to docs.google.com (docs.google.com)|216.58.211.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/trjeh51b241a24lkos29aof15c/1744895075000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx [following]
--2025-04-17 13:04:36--  https://doc-0k-3k-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/trjeh51b241a24lkos29aof15c/1744895075000/113369956649761615778/*/1riSLCvdlQ8kC5DshtqlRAsVEa7R7N2kD?format=xlsx
Resolving doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercontent.com)... 216.58.211.225, 2a00:1450:4026:808::2001
Connecting to doc-0k-3k-sheets.googleusercontent.com (doc-0k-3k-sheets.googleusercont

In [61]:
import pandas as pd

In [ ]:
df = pd.read_excel('/tmp/prices_scans.xlsx', header=1)

In [64]:
df.set_index("Номер картинки",  inplace=True)

In [10]:
df.head(4)

,Цена правильная,Цена нейросеть,Ошибка,Строка 1,Строка 2,Строка 3,Строка 4,Строка 5
Номер картинки,,,,,,,,
1,54.99,54.99,0.0,Вода СВЯТОЙ ИСТОЧНИК,1.0л,природная питьевая негаз.ПЭТ,NaN,NaN
2,59.99,59.99,0.0,Вода Святой источник,"0,75л",негазированная спорт,NaN,NaN
3,51.99,51.99,0.0,Вода ШИШКИН ЛЕС,питьевая 1л,негазированная,NaN,NaN
4,24.99,24.99,0.0,Вода КРАСНАЯ ЦЕНА,"1,5л",негазированная,NaN,NaN


In [65]:
import re
from collections import Counter

def generate_words_from_phrases(phrases, len_min=1, do_sort=True):
    word_counter = Counter()

    for phrase in phrases:
        if not isinstance(phrase, str):
            continue  # пропускаем нестроковые значения

        
        words = re.split(r"[ \t\n\r\f\v.,!?;:()\"«»—–\-\/]+", phrase.upper().replace("Ё", "Е"))
        words = list(filter(None, words))  # убираем пустые строки, если они есть
        
        # Оставляем только слова без цифр
        filtered_words = [
            word for word in words
            if len(word) > len_min and not re.search(r"\d", word)
        ]
        word_counter.update(filtered_words)

    if do_sort:
        return sorted(word_counter.items(), key=lambda x: x[1], reverse=True)
    else:
        return word_counter.items()


In [66]:
# Здесь у нас все фразы из таблицы
phrases = pd.concat([df["Строка 1"].dropna(), df["Строка 2"].dropna(), df["Строка 3"].dropna()]).astype(str).unique()


In [67]:
sorted_word_counters = generate_words_from_phrases(phrases)


In [74]:
new_df = pd.DataFrame(sorted_word_counters, columns=["word", "count"])
new_df

,word,count
0,ИЗ,46
1,СО,35
2,ДЛЯ,30
3,ДЕТ,27
4,ПЭТ,26
...,...,...
1793,КАРЕНАЯ,1
1794,РОССИЙСКИЙ,1
1795,РАЖЕ,1
1796,ШАРИКИ,1


In [75]:
new_df.drop_duplicates(subset=['word'], inplace=True)


In [81]:
new_df[new_df['word'].str.contains("фруто", case=False, na=False)]

,word,count
41,ФРУТОНЯНЯ,11
88,ФРУТО,6


In [ ]:
del(new_df['count'])
new_df.reset_index(drop=True, inplace=True)
new_df.to_csv('/home/jovyan/work/words.txt', index=False, encoding='utf-8', header=False)